In [1]:
import numpy as np
import pandas as pd
import random
import string

from sklearn.model_selection import ParameterSampler
from scipy.stats import randint as sp_randint
from scipy.stats import uniform, poisson

In [2]:
# city names
canada_cities = pd.read_csv('list-of-cities-in-canada-csv.csv',names=['city'])
canada_cities = list(canada_cities['city'])

#make of cars
car_make = [
    'honda',
    'toyota',
    'bmw',
    'mercedes',
    'ford',
    'gmc',
    'tesla',
    'chevy',
    'audi',
    'nissan',
    'acura',
]

model_dict = {
    'honda': ['civic', 'accord'],
    'toyota': ['camry', 'corrola'],
    'bmw': ['m3', 'i8', 'x5'],
    'mercedes': ['c', 's', 'g'],
    'ford': ['f150', 'focus', 'explorer'],
    'gmc': ['terrain','sierra','yukon'],
    'tesla': ['x', '3', 's'],
    'chevy': ['equinox', 'malibu'],
    'audi': ['r8','e-tron'],
    'nissan':['mirano','micro'],
    'acura':['rsx', 'mdx']
}


In [3]:
parameters = {'age':  sp_randint(18, 85),
              'city': canada_cities,
              'year': sp_randint(1998, 2020),
              'make': car_make,
              'dist': sp_randint(1200, 25000),
              'gender': ['m','f'],
              }

# other columns
    # new_used
    # model

p_list = list(ParameterSampler(parameters, n_iter=10,random_state=32))

In [4]:
df = pd.DataFrame(p_list)
df

,age,city,dist,gender,make,year
0,61,Belleville,10726,m,audi,2017
1,21,Wood Buffalo,9401,m,toyota,2001
2,83,Kelowna,9427,m,acura,2016
3,28,St. John,21090,m,ford,2003
4,56,Sarnia,7099,m,audi,2018
5,82,Burnaby,6942,m,ford,2007
6,48,Sherbrooke,11651,m,gmc,2008
7,53,Peterborough,23266,f,audi,2013
8,29,Vancouver,12383,f,bmw,2000
9,66,St. John,13954,f,acura,2000


In [5]:
def select_model_and_age(df, model_dict):
    model_list = []
    age_list = []
    
    for i in df.itertuples():
        if i.make in list(model_dict.keys()):
            model_ind = random.choice(model_dict[i.make])
            model_list.append(model_ind)
        else:
            model_list.append('other')
            
        if i.year > 2018:
            age_list.append('new')
        else:
            age_list.append('used')
            
        
    
        
            
    df['model'] = model_list
    df['new_used'] = age_list
    return df
    

In [6]:
df = select_model_and_age(df, model_dict)
df

,age,city,dist,gender,make,year,model,new_used
0,61,Belleville,10726,m,audi,2017,r8,used
1,21,Wood Buffalo,9401,m,toyota,2001,corrola,used
2,83,Kelowna,9427,m,acura,2016,mdx,used
3,28,St. John,21090,m,ford,2003,f150,used
4,56,Sarnia,7099,m,audi,2018,e-tron,used
5,82,Burnaby,6942,m,ford,2007,focus,used
6,48,Sherbrooke,11651,m,gmc,2008,sierra,used
7,53,Peterborough,23266,f,audi,2013,e-tron,used
8,29,Vancouver,12383,f,bmw,2000,i8,used
9,66,St. John,13954,f,acura,2000,rsx,used


In [7]:
df

,age,city,dist,gender,make,year,model,new_used
0,61,Belleville,10726,m,audi,2017,r8,used
1,21,Wood Buffalo,9401,m,toyota,2001,corrola,used
2,83,Kelowna,9427,m,acura,2016,mdx,used
3,28,St. John,21090,m,ford,2003,f150,used
4,56,Sarnia,7099,m,audi,2018,e-tron,used
5,82,Burnaby,6942,m,ford,2007,focus,used
6,48,Sherbrooke,11651,m,gmc,2008,sierra,used
7,53,Peterborough,23266,f,audi,2013,e-tron,used
8,29,Vancouver,12383,f,bmw,2000,i8,used
9,66,St. John,13954,f,acura,2000,rsx,used


In [27]:
car_make = [
    'honda',
    'toyota',
    'bmw',
    'mercedes',
    'ford',
    'gmc',
    'tesla',
    'chevy',
    'audi',
    'nissan',
    'acura',
]

In [36]:
def insurance_price(df, base_rate, weights):
    weight_age = weights['weight_age']
    weight_city = weights['weight_city']
    weight_year = weights['weight_year']
    weight_make = weights['weight_make']
    weight_dist = weights['weight_dist']
    weight_gender = weights['weight_gender']
    rate_age = weights['rate_age']
    rate_make = weights['rate_make']
    rate_city = weights['rate_city']
    rate_year = weights['rate_year']
    rate_dist = weights['rate_dist']
    rate_gender = weights['rate_gender']
    
    # cost = base_rate + (weight_age * rate_age * age / 100) + weight_city * rate_city * category_cost+
    # weight_make * rate_make * category_cost + (weight_dist * dist * rate_dist)/1000 + 
    # weight_gender * rate_gender
    l = []
    for i in df.itertuples():
        # age buckets
        if i.age < 30:
            age_category_cost = 100
        elif i.age >=30 < 65:
            age_category_cost = 20
        elif i.age >=65 < 75:
            age_category_cost = 40
        else:
            age_category_cost = 120
        
        # city buckets
        city_index = string.ascii_lowercase.index(i.city[0].lower())
        if city_index < 10:
            city_category_cost = 10
        elif city_index >=10 < 17:
            city_category_cost = 50
        else:
            city_category_cost = 80
            
        
        # make buckets
        if i.make in ['honda', 'toyota','gmc','nissan','ford','chevy']:
            make_category_cost = 50
        elif i.make in ['acura', 'tesla']:
            make_category_cost = 90
        elif i.make in ['mercedes','bmw', 'audi']:
            make_category_cost = 140
        else:
            make_category_cost = 60
            
        # gender buckets
        if i.gender == 'm':
            gender_category_cost = 100
        else:
            gender_category_cost = 50
        
        # distance buckets
        if i.dist <3000:
            dist_category_cost = 50
        elif i.dist >=3000 < 8000:
            dist_category_cost = 100
        else:
            dist_category_cost = 160

            
        # calculation for making the cost
        cost_total = (base_rate + (weight_age * age_category_cost) ** rate_age  + (weight_city * city_category_cost) ** rate_city + (weight_make * make_category_cost) ** rate_make + (weight_dist * dist_category_cost) ** rate_dist + (weight_gender * gender_category_cost) ** rate_gender)
        
        print(cost_total)
        
        l.append(cost_total)
    df['quote'] = l
    return df

In [40]:
weights = {
    "weight_age": 1.5,
    "weight_city": 0.5,
    "weight_year": 1.2,
    "weight_make": 0.7,
    "weight_dist": 2,
    "weight_gender": 1,
    "rate_age": 1.07,
    "rate_city": 1.09,
    "rate_make": 1.06,
    "rate_year": 1,
    "rate_dist": 1,
    "rate_gender": 1.04,
}

In [41]:
df = insurance_price(df, 100, weights)

20

593.1028940441764
100

709.9680910897167
20

572.4710026187604
100

709.9680910897167
20

620.7241927816491
20

507.392599043216
20

535.0138977806887
20

558.9671718413309
100

733.9213651503588
20

510.7139816784421


In [42]:
df.head()

,age,city,dist,gender,make,year,model,new_used,quote
0,61,Belleville,10726,m,audi,2017,r8,used,593.102894
1,21,Wood Buffalo,9401,m,toyota,2001,corrola,used,709.968091
2,83,Kelowna,9427,m,acura,2016,mdx,used,572.471003
3,28,St. John,21090,m,ford,2003,f150,used,709.968091
4,56,Sarnia,7099,m,audi,2018,e-tron,used,620.724193


In [22]:
a = 'Toronto'
string.ascii_lowercase.index(a[0].lower())

19

In [19]:
a[0]

'T'

In [21]:
string.ascii_lowercase.index('t')

19